# Import Event Log

In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the XES file
    dataframe_log = pm4py.read_xes('../../data/logs/PrepaidTravelCost.xes')

    # If 'log' is already a DataFrame, add the @@case_index column directly
    case_indices = {case_id: idx for idx, case_id in enumerate(dataframe_log['case:concept:name'].unique())}
    dataframe_log['@@case_index'] = dataframe_log['case:concept:name'].map(case_indices)
    
     # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)
    
dataframe_log

/opt/anaconda3/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:76: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/2099 [00:00<?, ?it/s]

,id,org:resource,concept:name,time:timestamp,org:role,case:Rfp_id,case:Permit travel permit number,case:Task,case:OrganizationalEntity,case:RequestedAmount,...,case:Permit ProjectNumber,case:Project,case:concept:name,case:Permit OrganizationalEntity,case:Permit RequestedBudget,case:Cost Type,case:Permit id,case:Permit ActivityNumber,case:RfpNumber,@@case_index
0,st_step 73555_0,STAFF MEMBER,Permit SUBMITTED by EMPLOYEE,2017-01-09 13:48:43+00:00,EMPLOYEE,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551,0
1,st_step 73554_0,STAFF MEMBER,Permit FINAL_APPROVED by SUPERVISOR,2017-01-09 13:48:55+00:00,SUPERVISOR,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551,0
2,st_step 73558_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-01-12 10:40:27+00:00,EMPLOYEE,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551,0
3,st_step 73559_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2017-01-12 10:41:59+00:00,SUPERVISOR,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551,0
4,st_step 73557_0,STAFF MEMBER,Request For Payment REJECTED by MISSING,2017-01-12 10:53:07+00:00,MISSING,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18241,st_step 186614_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2018-12-30 19:16:15+00:00,EMPLOYEE,request for payment 186612,UNKNOWN,UNKNOWN,organizational unit 65462,96.739075,...,UNKNOWN,project 147531,request for payment 186612,UNKNOWN,0.000000,0,UNKNOWN,UNKNOWN,request for payment number 186613,2098
18242,st_step 186616_0,STAFF MEMBER,Request For Payment APPROVED by ADMINISTRATION,2018-12-30 19:16:25+00:00,ADMINISTRATION,request for payment 186612,UNKNOWN,UNKNOWN,organizational unit 65462,96.739075,...,UNKNOWN,project 147531,request for payment 186612,UNKNOWN,0.000000,0,UNKNOWN,UNKNOWN,request for payment number 186613,2098
18243,st_step 186615_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2019-01-14 14:09:11+00:00,SUPERVISOR,request for payment 186612,UNKNOWN,UNKNOWN,organizational unit 65462,96.739075,...,UNKNOWN,project 147531,request for payment 186612,UNKNOWN,0.000000,0,UNKNOWN,UNKNOWN,request for payment number 186613,2098
18244,rp_request for payment 186612_15,SYSTEM,Request Payment,2019-01-15 06:02:45+00:00,UNDEFINED,request for payment 186612,UNKNOWN,UNKNOWN,organizational unit 65462,96.739075,...,UNKNOWN,project 147531,request for payment 186612,UNKNOWN,0.000000,0,UNKNOWN,UNKNOWN,request for payment number 186613,2098


# Drop unnessary columns

In [2]:
dataframe_log = dataframe_log.drop(columns=['case:Rfp_id'])

In [3]:
dataframe_log = dataframe_log.drop(columns=['case:Permit travel permit number'])

In [4]:
dataframe_log = dataframe_log.drop(columns=['case:Task'])

In [5]:
dataframe_log = dataframe_log.drop(columns=['case:OrganizationalEntity'])

In [6]:
dataframe_log = dataframe_log.drop(columns=['case:RequestedAmount'])

In [7]:
dataframe_log = dataframe_log.drop(columns=['case:Permit TaskNumber'])

In [8]:
dataframe_log = dataframe_log.drop(columns=['case:Activity'])

In [9]:
dataframe_log = dataframe_log.drop(columns=['case:Permit BudgetNumber'])

In [10]:
dataframe_log = dataframe_log.drop(columns=['case:Permit ProjectNumber'])

In [11]:
dataframe_log = dataframe_log.drop(columns=['case:Project'])

In [12]:
dataframe_log = dataframe_log.drop(columns=['case:concept:name'])

In [13]:
dataframe_log = dataframe_log.drop(columns=['case:Permit OrganizationalEntity'])

In [14]:
dataframe_log = dataframe_log.drop(columns=['case:Permit RequestedBudget'])

In [15]:
dataframe_log = dataframe_log.drop(columns=['case:Cost Type'])

In [16]:
dataframe_log = dataframe_log.drop(columns=['case:Permit id'])

In [17]:
dataframe_log = dataframe_log.drop(columns=['case:Permit ActivityNumber'])

In [18]:
dataframe_log = dataframe_log.drop(columns=['case:RfpNumber'])

In [19]:
from sklearn.preprocessing import StandardScaler

# Convert to datetime format
dataframe_log['time:timestamp'] = pd.to_datetime(dataframe_log['time:timestamp'])

# Calculate elapsed time since the start of each case
dataframe_log['start_time'] = dataframe_log.groupby('@@case_index')['time:timestamp'].transform('min')
dataframe_log['elapsed_time'] = (dataframe_log['time:timestamp'] - dataframe_log['start_time']).dt.total_seconds()

# Normalize the elapsed time in minutes
scaler = StandardScaler()
dataframe_log['standardized_elapsed_time'] = scaler.fit_transform(dataframe_log[['elapsed_time']])

dataframe_log = dataframe_log.drop(columns=['start_time'])
dataframe_log = dataframe_log.drop(columns=['elapsed_time'])
dataframe_log = dataframe_log.drop(columns=['time:timestamp'])

In [20]:
dataframe_log

,id,org:resource,concept:name,org:role,@@case_index,standardized_elapsed_time
0,st_step 73555_0,STAFF MEMBER,Permit SUBMITTED by EMPLOYEE,EMPLOYEE,0,-0.601890
1,st_step 73554_0,STAFF MEMBER,Permit FINAL_APPROVED by SUPERVISOR,SUPERVISOR,0,-0.601886
2,st_step 73558_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,EMPLOYEE,0,-0.514864
3,st_step 73559_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,SUPERVISOR,0,-0.514832
4,st_step 73557_0,STAFF MEMBER,Request For Payment REJECTED by MISSING,MISSING,0,-0.514598
...,...,...,...,...,...,...
18241,st_step 186614_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,EMPLOYEE,2098,-0.601890
18242,st_step 186616_0,STAFF MEMBER,Request For Payment APPROVED by ADMINISTRATION,ADMINISTRATION,2098,-0.601886
18243,st_step 186615_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,SUPERVISOR,2098,-0.153403
18244,rp_request for payment 186612_15,SYSTEM,Request Payment,UNDEFINED,2098,-0.133318


# Insert Start & End markers

In [21]:
# Define a function to insert start and end markers
def add_markers(df):
    # Identify unique case indices
    case_indices = df['@@case_index'].unique()
    
    # Prepare a container for new DataFrame rows
    new_rows = []
    
    # Iterate over each case index to add start and end markers
    for case_index in case_indices:
        # Create a start marker row with all columns except @@case_index set to 'Start'
        start_row = {col: 'Start' if col != '@@case_index' else case_index for col in df.columns}
        
        # Create an end marker row with all columns except @@case_index set to 'End'
        end_row = {col: 'End' if col != '@@case_index' else case_index for col in df.columns}
        
        # Append start row, rows for the current case, and end row
        new_rows.append(start_row)
        new_rows.extend(df[df['@@case_index'] == case_index].to_dict('records'))
        new_rows.append(end_row)
    
    # Convert the list of rows into a DataFrame
    return pd.DataFrame(new_rows)

# Apply the function to add start and end markers to the dataframe
modified_dataframe = add_markers(dataframe_log)

# Preprocess

In [22]:
modified_dataframe['standardized_elapsed_time'] = modified_dataframe['standardized_elapsed_time'].replace({'Start': 0, 'End': 1})

In [23]:
codes, uniques = pd.factorize(modified_dataframe['id'])
modified_dataframe['id'] = codes

In [24]:
codes, uniques = pd.factorize(modified_dataframe['org:resource'])
modified_dataframe['org:resource'] = codes

In [25]:
codes, uniques = pd.factorize(modified_dataframe['concept:name'])
modified_dataframe['concept:name'] = codes

In [26]:
codes, uniques = pd.factorize(modified_dataframe['org:role'])
modified_dataframe['org:role'] = codes

### Padding for Cases with less then 5 events

In [27]:
# Calculating the frequency of each unique value in '@@case_index'
frequency = modified_dataframe['@@case_index'].value_counts()

# Finding the minimum occurrence
min_occurrence = frequency.min()

min_occurrence

3

In [28]:
# Assuming your dataframe might have different types, let's create a generic function to add rows
def add_rows(group, num_rows, case_index_value):
    # For each column, determine the appropriate "zero" value (int 0, string '', etc.)
    additional_rows = pd.DataFrame({
        column: 0 if pd.api.types.is_numeric_dtype(group[column]) else '' for column in group.columns
    }, index=range(num_rows))
    
    # Set the @@case_index column to the current case index value
    additional_rows['@@case_index'] = case_index_value
    
    # Append the additional rows to the group
    return pd.concat([group, additional_rows], ignore_index=True)

# Function to pad cases with less than 5 events
def pad_cases(df):
    # Group by @@case_index
    groups = df.groupby('@@case_index')
    
    # Placeholder for modified groups
    modified_groups = []
    
    for name, group in groups:
        # Calculate the number of events to add
        events_to_add = 5 - len(group)
        
        if events_to_add > 0:
            # Add the required number of rows
            group = add_rows(group, events_to_add, name)
        
        # Append the modified group to the list
        modified_groups.append(group)
    
    # Concatenate all modified groups back into a single DataFrame
    return pd.concat(modified_groups, ignore_index=True)

In [29]:
# Apply the padding function
modified_dataframe = pad_cases(modified_dataframe)
modified_dataframe

,id,org:resource,concept:name,org:role,@@case_index,standardized_elapsed_time
0,0,0,0,0,0,0.000000
1,1,1,1,1,0,-0.601890
2,2,1,2,2,0,-0.601886
3,3,1,3,1,0,-0.514864
4,4,1,4,2,0,-0.514832
...,...,...,...,...,...,...
22453,16203,1,23,8,2098,-0.601886
22454,16204,1,4,2,2098,-0.153403
22455,16205,3,8,5,2098,-0.133318
22456,16206,3,9,5,2098,-0.059410


# Generate sliding windows

In [30]:
df_id = modified_dataframe[['id', '@@case_index']]
df_resource = modified_dataframe[['org:resource', '@@case_index']]
df_activity = modified_dataframe[['concept:name', '@@case_index']]
df_role = modified_dataframe[['org:role', '@@case_index']]
df_timestamp = modified_dataframe[['standardized_elapsed_time', '@@case_index']]

In [31]:
def generate_sliding_windows(df, case_id_column='@@case_index', window_size=5):
    windows = []
    targets = []
    case_indices = []

    # Iterate over each unique case
    for case_id in df[case_id_column].unique():
        # Extract the case
        case_data = df[df[case_id_column] == case_id]
        
        # Convert case_data to a NumPy array and drop the case_id_column
        case_data_array = case_data.drop(columns=[case_id_column]).to_numpy()

        # Adjusting the condition to correctly reflect window_size without needing an additional +1
        # Now it correctly considers window_size as including the target event
        if len(case_data_array) >= window_size:
            # Adjust the loop to generate sliding windows of size window_size - 1 for the inputs and use the next event as the target
            for i in range(len(case_data_array) - window_size + 1):
                # window now has window_size - 1 events
                window = case_data_array[i:i + window_size - 1]
                # The target is the event immediately following the window
                target = case_data_array[i + window_size - 1]
                windows.append(window)
                targets.append(target)
                case_indices.append(case_id)  # Store the case_id corresponding to the window

    # Convert lists to numpy arrays for easier handling and to ensure they are two-dimensional
    windows_array = np.array(windows)
    targets_array = np.array(targets)
    case_indices_array = np.array(case_indices)
    
    return windows_array, targets_array, case_indices_array

In [32]:
windows_id, targets_id, case_indices = generate_sliding_windows(df_id)
windows_resource, targets_resource, case_indices = generate_sliding_windows(df_resource)
windows_activity, targets_activity, case_indices = generate_sliding_windows(df_activity)
windows_role, targets_role, case_indices = generate_sliding_windows(df_role)
windows_timestamp, targets_timestamp, case_indices = generate_sliding_windows(df_timestamp)


# LSTM

### Architecture

In [33]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Embedding

# Assuming these values as placeholders, replace them with actual counts from your data
num_ids = modified_dataframe['id'].nunique()
num_resources = modified_dataframe['org:resource'].nunique()
num_activities = modified_dataframe['concept:name'].nunique()
num_roles = modified_dataframe['org:role'].nunique()

embedding_dim_id = 50
embedding_dim_resource = 50
embedding_dim_activity = 50
embedding_dim_role = 50

time_steps = 4

# Input layers
input_id = Input(shape=(time_steps,), name='input_id')
input_resource = Input(shape=(time_steps,), name='input_resource')
input_activity = Input(shape=(time_steps,), name='input_activity')
input_role = Input(shape=(time_steps,), name='input_role')
input_timestamp = Input(shape=(time_steps, 1), name='input_timestamp')

# Embedding layers
embedding_id = Embedding(input_dim=num_ids, output_dim=embedding_dim_id, input_length=time_steps)(input_id)
embedding_resource = Embedding(input_dim=num_resources, output_dim=embedding_dim_resource, input_length=time_steps)(input_resource)
embedding_activity = Embedding(input_dim=num_activities, output_dim=embedding_dim_activity, input_length=time_steps)(input_activity)
embedding_role = Embedding(input_dim=num_roles, output_dim=embedding_dim_role, input_length=time_steps)(input_role)

# LSTM layers
lstm_id = LSTM(25, return_sequences=False)(embedding_id)
lstm_resource = LSTM(25, return_sequences=False)(embedding_resource)
lstm_activity = LSTM(25, return_sequences=False)(embedding_activity)
lstm_role = LSTM(25, return_sequences=False)(embedding_role)
lstm_timestamp = LSTM(25, return_sequences=False)(input_timestamp)

# Concatenate outputs
concatenated = Concatenate(axis=-1)([lstm_id, lstm_resource, lstm_activity, lstm_role, lstm_timestamp])

# Output layers
output_id = Dense(num_ids, activation='softmax', name='output_id')(concatenated)
output_resource = Dense(num_resources, activation='softmax', name='output_resource')(concatenated)
output_activity = Dense(num_activities, activation='softmax', name='output_activity')(concatenated)
output_role = Dense(num_roles, activation='softmax', name='output_role')(concatenated)
output_timestamp = Dense(1, activation='linear', name='output_timestamp')(concatenated)

# Create and compile the model
model = Model(inputs=[input_id, input_resource, input_activity, input_role, input_timestamp], 
              outputs=[output_id, output_resource, output_activity, output_role, output_timestamp])

model.compile(optimizer='adam', 
              loss={'output_id': 'categorical_crossentropy', 
                    'output_resource': 'categorical_crossentropy', 
                    'output_activity': 'categorical_crossentropy', 
                    'output_role': 'categorical_crossentropy', 
                    'output_timestamp': 'mean_squared_error'},
              metrics={'output_id': 'accuracy', 
                       'output_resource': 'accuracy', 
                       'output_activity': 'accuracy', 
                       'output_role': 'accuracy', 
                       'output_timestamp': 'mean_absolute_error'})

model.summary()


2024-08-19 22:04:56.791270: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_id (InputLayer)       [(None, 4)]                  0         []                            
                                                                                                  
 input_resource (InputLayer  [(None, 4)]                  0         []                            
 )                                                                                                
                                                                                                  
 input_activity (InputLayer  [(None, 4)]                  0         []                            
 )                                                                                                
                                                                                              

# Data Splitting

In [34]:
from sklearn.model_selection import train_test_split

# Split the id data
train_id, test_id, train_targets_id, test_targets_id = train_test_split(
    windows_id, targets_id, test_size=0.3, random_state=42)

# Split the resource data
train_resource, test_resource, train_targets_resource, test_targets_resource = train_test_split(
    windows_resource, targets_resource, test_size=0.3, random_state=42)

# Split the activity data
train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

# Split the role data
train_role, test_role, train_targets_role, test_targets_role = train_test_split(
    windows_role, targets_role, test_size=0.3, random_state=42)

# Split the timestamp data
train_timestamp, test_timestamp, train_targets_timestamp, test_targets_timestamp = train_test_split(
    windows_timestamp, targets_timestamp, test_size=0.3, random_state=42)


# Training

In [35]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler

# EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

def cyclic_lr(epoch, lr):
    # Example function that modulates LR within a range for each epoch
    # Customize this function based on your cyclic learning rate policy
    max_lr = 0.01  # Maximum LR
    base_lr = 0.001  # Base LR
    step_size = 10  # Number of epochs for half a cycle
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * np.maximum(0, (1 - x))
    return lr

lr_scheduler = LearningRateScheduler(cyclic_lr)

In [36]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Convert categorical targets to one-hot encoding
train_targets_id_cat = to_categorical(train_targets_id, num_classes=num_ids)
test_targets_id_cat = to_categorical(test_targets_id, num_classes=num_ids)

train_targets_resource_cat = to_categorical(train_targets_resource, num_classes=num_resources)
test_targets_resource_cat = to_categorical(test_targets_resource, num_classes=num_resources)

train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities)

train_targets_role_cat = to_categorical(train_targets_role, num_classes=num_roles)
test_targets_role_cat = to_categorical(test_targets_role, num_classes=num_roles)

# Note: Assuming timestamp targets are continuous and don't need to be converted to categorical

# Train the model
history = model.fit([train_id, train_resource, train_activity, train_role, train_timestamp], 
                    [train_targets_id_cat, train_targets_resource_cat, train_targets_activity_cat, train_targets_role_cat, train_targets_timestamp],
                    epochs=25,
                    batch_size=64,
                    validation_data=([test_id, test_resource, test_activity, test_role, test_timestamp], 
                                     [test_targets_id_cat, test_targets_resource_cat, test_targets_activity_cat, test_targets_role_cat, test_targets_timestamp]),
                    verbose=1,
                    callbacks=[lr_scheduler])  # Add other callbacks as needed

Epoch 1/25
154/154 [==============================] - 18s 53ms/step - loss: 14.6182 - output_id_loss: 9.3990 - output_resource_loss: 0.6611 - output_activity_loss: 2.2667 - output_role_loss: 1.5429 - output_timestamp_loss: 0.7485 - output_id_accuracy: 0.1398 - output_resource_accuracy: 0.7566 - output_activity_accuracy: 0.4603 - output_role_accuracy: 0.4884 - output_timestamp_mean_absolute_error: 0.5595 - val_loss: 11.9222 - val_output_id_loss: 8.9149 - val_output_resource_loss: 0.1971 - val_output_activity_loss: 1.3853 - val_output_role_loss: 0.9169 - val_output_timestamp_loss: 0.5079 - val_output_id_accuracy: 0.1413 - val_output_resource_accuracy: 0.9746 - val_output_activity_accuracy: 0.7096 - val_output_role_accuracy: 0.7139 - val_output_timestamp_mean_absolute_error: 0.4366 - lr: 0.0010
Epoch 2/25
154/154 [==============================] - 5s 33ms/step - loss: 9.6627 - output_id_loss: 7.8638 - output_resource_loss: 0.1019 - output_activity_loss: 0.8274 - output_role_loss: 0.5162 -

In [37]:
# Evaluate the model
evaluation = model.evaluate(
    [test_id, test_resource, test_activity, test_role, test_timestamp],
    [test_targets_id_cat, test_targets_resource_cat, test_targets_activity_cat, test_targets_role_cat, test_targets_timestamp]
)

print(evaluation)

132/132 [==============================] - 2s 11ms/step - loss: 17.3264 - output_id_loss: 15.5437 - output_resource_loss: 0.1591 - output_activity_loss: 0.8050 - output_role_loss: 0.4550 - output_timestamp_loss: 0.3636 - output_id_accuracy: 0.1818 - output_resource_accuracy: 0.9775 - output_activity_accuracy: 0.8689 - output_role_accuracy: 0.9324 - output_timestamp_mean_absolute_error: 0.3885
[17.326356887817383, 15.543675422668457, 0.1590566784143448, 0.8049982190132141, 0.4550149738788605, 0.36361393332481384, 0.18179664015769958, 0.977482795715332, 0.8689262866973877, 0.9324484467506409, 0.38850152492523193]


In [38]:
# Save the model to an H5 file
model.save('dapnn_prepaid.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

In [39]:
# Generate predictions for all inputs
predictions = model.predict([windows_id, windows_resource, windows_activity, windows_role, windows_timestamp])

# Extract predictions for categorical attributes (softmax probabilities)
predictions_id = predictions[0]         # ID predictions
predictions_resource = predictions[1]   # Resource predictions
predictions_activity = predictions[2]   # Activity predictions
predictions_role = predictions[3]       # Role predictions

# Extract predictions for numerical attribute (timestamp)
predictions_timestamp = predictions[4]  # Timestamp predictions

440/440 [==============================] - 6s 8ms/step


In [40]:
import numpy as np
from tensorflow.keras.utils import to_categorical

def compute_anomaly_scores(predictions, actuals):
    # For categorical predictions, convert actuals to one-hot for comparison
    actuals_one_hot = to_categorical(actuals, num_classes=predictions.shape[-1])
    
    max_predictions = np.max(predictions, axis=-1)
    actual_predictions = np.sum(predictions * actuals_one_hot, axis=-1)  # Extract the probability of the actual class
    
    anomaly_scores = (max_predictions - actual_predictions) / max_predictions
    
    return anomaly_scores

# Assuming targets_id, targets_resource, targets_activity, targets_role are the true values for these attributes
anomaly_scores_id = compute_anomaly_scores(predictions_id, targets_id)
anomaly_scores_resource = compute_anomaly_scores(predictions_resource, targets_resource)
anomaly_scores_activity = compute_anomaly_scores(predictions_activity, targets_activity)
anomaly_scores_role = compute_anomaly_scores(predictions_role, targets_role)

In [41]:
def compute_anomaly_scores_continuous(predictions, actuals, normalization_factor):
    """
    Compute anomaly scores for continuous attributes.
    
    Parameters:
    - predictions: numpy array of predicted values.
    - actuals: numpy array of actual values.
    - normalization_factor: normalization factor (e.g., standard deviation of the attribute).
    
    Returns:
    - numpy array of anomaly scores.
    """
    # Calculate absolute differences
    differences = np.abs(predictions - actuals)
    
    # Normalize the differences
    anomaly_scores = differences / normalization_factor
    
    return anomaly_scores

normalization_factor = np.std(targets_timestamp)       # Example normalization factor (standard deviation)
anomaly_scores_timestamp = compute_anomaly_scores_continuous(predictions_timestamp, targets_timestamp, normalization_factor)

In [42]:
import numpy as np

def classify_cases(anomaly_scores_resource, anomaly_scores_activity, anomaly_scores_role, anomaly_scores_timestamp, threshold=0.98):
    # Ensure all inputs are numpy arrays of the same shape
    anomaly_scores_resource = np.array(anomaly_scores_resource).flatten()
    anomaly_scores_activity = np.array(anomaly_scores_activity).flatten()
    anomaly_scores_role = np.array(anomaly_scores_role).flatten()
    anomaly_scores_timestamp = np.array(anomaly_scores_timestamp).flatten()

    # Check if all arrays have the same length
    if not (len(anomaly_scores_resource) == len(anomaly_scores_activity) == len(anomaly_scores_role) == len(anomaly_scores_timestamp)):
        raise ValueError("All input anomaly scores must have the same length.")

    # Find the maximum anomaly score across all attributes for each case
    max_scores = np.maximum.reduce([anomaly_scores_id, anomaly_scores_resource, anomaly_scores_activity, anomaly_scores_role, anomaly_scores_timestamp])

    # Classify cases as anomalous if the maximum anomaly score exceeds the threshold
    anomalous_cases = max_scores > threshold
    
    return anomalous_cases

# Now use the anomaly scores for id, resource, activity, role, and timestamp in the classification
anomalous_cases = classify_cases(anomaly_scores_resource, anomaly_scores_activity, anomaly_scores_role, anomaly_scores_timestamp)

# Mapping

### True: anomaly, False: no anomaly

In [43]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted'] = anomalous_cases
mapping

,case,predicted
0,0,True
1,0,False
2,0,False
3,0,True
4,1,False
...,...,...
14057,2097,False
14058,2097,False
14059,2098,False
14060,2098,True


In [44]:
case_prediction = mapping.groupby('case')['predicted'].any()
case_prediction


case
0        True
1        True
2       False
3        True
4       False
        ...  
2094     True
2095     True
2096     True
2097    False
2098     True
Name: predicted, Length: 2099, dtype: bool

# Ground Truth

In [45]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [46]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/Model_PrepaidTravelCost.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/202 [00:00<?, ?it/s]

In [47]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [48]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth

,conformity,predicted
0,1,1
1,1,1
2,1,0
3,1,1
4,1,0
...,...,...
2094,1,1
2095,1,1
2096,1,1
2097,1,0


# Evaluation

In [49]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [50]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 0.41


In [51]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 0.81


In [52]:
precision_no_dev = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_dev:.2f}")

Precision No Dev: 0.46


In [53]:
recall_no_dev = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_dev:.2f}")

Recall No Dev: 0.13


In [54]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.47
